In [1]:
# # !pip install opencv-python
# # !pip uninstall h5py
#!pip install h5py==2.10.0 
# # After installing h5py v2.10.0 restart the kernel
# # h5dict, custom_objects, compile

In [2]:
# !pip install pillow -U
# !pip install pathlib -U

In [3]:
import h5py
print(h5py.__version__)

2.10.0


In [4]:
import sys
from PIL import Image

In [5]:
## Import Packages
import cv2
import os
# from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np

Using TensorFlow backend.


In [6]:
## Load the pre-trained CNN model
model = load_model('models/resnet152v2.h5') 

# model accepts height and width of the image as spec below:
img_width, img_height = 224, 224

## Load the Cascade face Classifier
face_cascade = cv2.CascadeClassifier('cascade_classifier/haarcascade_frontalface_default.xml')

In [7]:
## Text font parameters
font = cv2.FONT_HERSHEY_SIMPLEX
org = (1, 1)
class_label = ' '
fontScale = 1
color = (255, 0 ,0) ## Blue col in BGR
thickness = 1  ## Line thickness of 1 px

In [8]:
## Read image from webcam
color_img = cv2.imread('face_mask_dataset/Test/WithoutMask/2044.png')
# 'Face-Mask-Detection-master/dataset/with_mask/0_0_0 copy 12.jpg'
# Convert to grayscale
gray_img = cv2.cvtColor(color_img, cv2.COLOR_BGR2GRAY)

# detect the face
faces = face_cascade.detectMultiScale(gray_img, 1.5, 3)

In [9]:
#take face then predict class mask or not mask then draw recrangle and text then display image

img_count = 0
for (x, y, w, h) in faces:
    org = (x-10, y-10)
    img_count += 1
    color_face = color_img[y:y+h, x:x+w] ## color face
    cv2.imwrite('face_mask_dataset/input/%dface.jpg'%(img_count), color_face)
    img = load_img('face_mask_dataset/input/%dface.jpg'%(img_count), target_size=(img_width, img_height))
    
    img = img_to_array(img)/255
    img = np.expand_dims(img, axis=0)
    pred_prob = model.predict(img)
    #print(pred_prob[0][0].round(2))
    pred=np.argmax(pred_prob)
    
    if pred == 0:
        print("User with mask - predict = ", pred_prob[0][0])
        class_label = "Mask"
        color = (255, 0, 0)
        cv2.imwrite('face_mask_dataset/with_masks/%dface.jpg'%(img_count), color_face)
        cv2.rectangle(color_img, (x,y), (x+w, y+h), (0, 0, 255), 3)
        cv2.putText(color_img, class_label, org, font, 
                    fontScale, color, thickness, cv2.LINE_AA)
        cv2.imwrite('face_mask_dataset/with_masks/%dmask.jpg'%(img_count), color_img)
    
    else:
        print("User not wearing mask - prob = ", pred_prob[0][1])
        class_label = "No Mask"
        color = (255, 0, 0)
        cv2.imwrite('face_mask_dataset/without_masks/%dface.jpg'%(img_count), color_face)
        cv2.rectangle(color_img, (x,y), (x+w, y+h), (0, 0, 255), 3)
        cv2.putText(color_img, class_label, org, font,
                        fontScale, color, thickness, cv2.LINE_AA)
        cv2.imwrite('face_mask_dataset/without_masks/%dnomask.jpg'%(img_count), color_img)